In [ ]:
pip install mysql-connector-python

In [ ]:
import mysql.connector
from mysql.connector import Error

def connect_to_db():
    try:
        connection = mysql.connector.connect(
            host="your_host_here",  # Replace with your database host, e.g., 'localhost'
            user="your_username_here",  # Replace with your database username
            password="your_password_here",  # Replace with your database password
            database="pekus",
            charset="utf8mb4",
            collation="utf8mb4_general_ci"
        )
        if connection.is_connected():
            print("Connected to the database.")
            return connection
    except Error as e:
        print(f"Error while connecting to the database: {e}")
        return None

In [ ]:
def get_user_data(connection, user_id):
    try:
        cursor = connection.cursor(dictionary=True)
        query = f"""
        SELECT id_user, genre, AVG(rating) AS average_rating
        FROM watched
        WHERE id_user = {user_id} AND finished = 'Yes'
        GROUP BY genre
        ORDER BY average_rating DESC
        LIMIT 1;
        """
        cursor.execute(query)
        result = cursor.fetchone()
        return result
    except Error as e:
        print(f"Error while fetching user data: {e}")
        return None

In [ ]:
def recommend_movie(connection, user_id, favorite_genre):
    try:
        cursor = connection.cursor(dictionary=True)
        query = f"""
        SELECT t.title
        FROM content t
        WHERE t.genre = '{favorite_genre}'
          AND t.show_id NOT IN (
              SELECT o.id_content
              FROM watched o
              WHERE o.id_user = {user_id}
          )
        ORDER BY RAND()
        LIMIT 1;
        """
        cursor.execute(query)
        result = cursor.fetchone()
        return result['title'] if result else "No available recommendations."
    except Error as e:
        print(f"Error while fetching recommendations: {e}")
        return "Error in recommendation."

In [ ]:
def main_recommendation_system(user_id):
    connection = connect_to_db()
    if connection:
        user_data = get_user_data(connection, user_id)
        if user_data:
            favorite_genre = user_data['genre']
            recommended_movie = recommend_movie(connection, user_id, favorite_genre)
            print(f"Recommended movie for user {user_id}: {recommended_movie}")
        else:
            print("No data found for this user.")
        connection.close()

In [ ]:
def get_favorite_genre(connection, user_id):
    """
    Fetches the user's favorite genre based on the highest average rating.
    """
    try:
        cursor = connection.cursor(dictionary=True)
        query = f"""
        SELECT genre, AVG(rating) AS average_rating
        FROM watched
        WHERE user_id = {user_id} AND finished = 'Yes'
        GROUP BY genre
        ORDER BY average_rating DESC
        LIMIT 1;
        """
        cursor.execute(query)
        result = cursor.fetchone()
        return result['genre'] if result else None
    except Error as e:
        print(f"Error while fetching favorite genre: {e}")
        return None

def recommend_movie(connection, user_id, favorite_genre):
    """
    Recommends a movie in the user's favorite genre that they haven't watched yet.
    """
    try:
        cursor = connection.cursor(dictionary=True)
        query = f"""
        SELECT t.title
        FROM content t
        WHERE t.genre = '{favorite_genre}'
          AND t.show_id NOT IN (
              SELECT o.content_id
              FROM watched o
              WHERE o.user_id = {user_id}
          )
        ORDER BY RAND()
        LIMIT 1;
        """
        cursor.execute(query)
        result = cursor.fetchone()
        return result['title'] if result else "No available recommendations."
    except Error as e:
        print(f"Error while recommending: {e}")
        return "Recommendation error."

def main_recommendation_system(user_id):
    """
    Main function handling the entire recommendation system.
    """
    connection = connect_to_db()
    if connection:
        favorite_genre = get_favorite_genre(connection, user_id)
        if favorite_genre:
            recommended_movie = recommend_movie(connection, user_id, favorite_genre)
            print(f"Recommended movie for user {user_id}: {recommended_movie}")
        else:
            print(f"Could not find the favorite genre for user {user_id}.")
        connection.close()
    else:
        print("No connection to the database.")

In [ ]:
main_recommendation_system(29)